<td>
   <a target="_blank" href="https://www.clarifai.com/" ><img src="https://upload.wikimedia.org/wikipedia/commons/b/bc/Clarifai_Logo_FC_Web.png" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Clarifai/examples/blob/main/models/model_train/image-detection_training.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab"></a>
</td>

# Models



Clarifai offers a range of powerful model types, each designed to generate meaningful outputs based on user specific inputs and AI tasks.

There are wide variety of models that can be used as standalone solutions, or as building blocks for your own custom business solutions.
 

Clarifai Models are the recommended starting points for many users because they offer incredibly fast training times when you customize them using the "embedding-classifier" (Transfer Learning Classifier) model type.

But there are many cases where accuracy and the ability to carefully target solutions take priority over speed and ease of use. Additionally, you may need a model to learn new features, not recognized by existing Clarifai Models. For these cases, it is possible to "deep fine-tune" your custom models and integrate them directly within your workflows.

You might consider deep training if you have:

- A custom tailored dataset
- Accurate labels
- Expertise and time to fine-tune models


_______
On the [Clarifai Community](https://clarifai.com/explore) explore page, you can click the [Models](https://clarifai.com/explore/models) tab to search and access the models available for everyone to use.

This notebook contains Model Train demo for **visual-detector** Model Type with **MMDetection** Template.

# Getting Started

### Installation

In [ ]:
! pip install clarifai

In [1]:
import os
os.environ["CLARIFAI_PAT"] = "PAT" # replace with your own PAT key here

*Note: Guide to get your [PAT](https://docs.clarifai.com/clarifai-basics/authentication/personal-access-tokens)*

### For Colab
To access data files from Clarifai examples repo, you can clone the repo

In [ ]:
!git clone https://github.com/Clarifai/examples.git
%cd /content/examples/

## VISUAL-DETECTION

**Input: Images and videos**

**Output: Regions**


Visual Detector, also known as Object Detection, is a type of deep fine-tuned model designed to identify and locate objects within images or video frames. It goes beyond simple image classification, where the goal is to assign a single label to an entire image.

Instead, an object detection model can identify multiple objects of different classes within an image and provide their corresponding bounding box coordinates.

### Creating an App

In [2]:
from clarifai.client.user import User
#replace your "user_id"
client = User(user_id="user_id")

In [ ]:
app = client.create_app(app_id="demo_train", base_workflow="Universal")

### Uploading Image Detection Dataset

#### Training Dataset

In [ ]:
module_path = os.path.join(os.getcwd().split('/models/model_train')[0],'datasets/upload/image_detection/voc')
module_path

In [6]:
#importing load_module_dataloader for calling the dataloader object in dataset.py in the local data folder
from clarifai.datasets.upload.utils import load_module_dataloader

In [7]:
voc_dataloader = load_module_dataloader(module_path)

In [ ]:
dataset = app.create_dataset(dataset_id="train_dataset")
dataset.upload_dataset(dataloader=voc_dataloader)

### List Trainable Model Types

In [7]:
app.list_trainable_model_types()

['visual-classifier',
 'visual-detector',
 'visual-segmenter',
 'visual-anomaly-heatmap',
 'visual-embedder',
 'clusterer',
 'text-classifier',
 'embedding-classifier',
 'text-to-text']

### Create a Model

In [ ]:
MODEL_ID = "model_detector"
MODEL_TYPE_ID = "visual-detector"
model = app.create_model(model_id=MODEL_ID, model_type_id=MODEL_TYPE_ID)

### List Templates for the Model Type

Templates give you the control to choose the specific architecture used by your neural network, as well as define a set of hyperparameters you can use to fine-tune the way your model learns.

In [9]:
model.list_training_templates()

['detection_msc10',
 'Clarifai_InceptionV2',
 'Clarifai_InceptionV4',
 'MMDetection_FasterRCNN',
 'MMDetection_SSD',
 'MMDetection',
 'MMDetection_YoloF',
 '_Ultralytics_YoloV5']

### Save params
Save the parameters for the specific model template in a YAML file

In [26]:
YAML_FILE = 'model_params.yaml'
model_params = model.get_params(template='MMDetection', save_to=YAML_FILE)

In [11]:
import yaml
# Preview YAML content
file = open(YAML_FILE)
data = yaml.safe_load(file)
data

{'dataset_id': '',
 'dataset_version_id': '',
 'concepts': [],
 'train_params': {'invalid_data_tolerance_percent': 5.0,
  'template': 'MMDetection',
  'seed': -1.0,
  'custom_config': 'MMDetection.py',
  'num_gpus': 1.0,
  'image_size': [320.0]},
 'inference_params': {'detection_threshold': 0.0}}

#### Get param info

In [ ]:
print(model.get_param_info(param = 'image_size'))

### Edit the parameters in the YAML file to pass it to model.train()
**Here using the already edited and saved YAML file**

In [46]:
# Preview YAML content
file = open('saved_mmdetection.yaml')
data = yaml.safe_load(file)
data

{'dataset_id': 'train_dataset',
 'dataset_version_id': '',
 'concepts': ['id-cow',
  'id-horse',
  'id-bottle',
  'id-sofa',
  'id-bird',
  'id-cat',
  'id-dog',
  'id-person'],
 'train_params': {'invalid_data_tolerance_percent': 5.0,
  'template': 'MMDetection',
  'seed': -1.0,
  'custom_config': 'saved_MMDetection.py',
  'num_gpus': 1.0,
  'image_size': [320.0]},
 'inference_params': {'detection_threshold': 0.0}}

## Custom Config

Clarifai training templates employ two training toolboxes: **HuggingFace and MMDetection.** 

These toolboxes offer various training templates that can be configured at great length by the ML Builder. Here an example for Visual Detection(via MMDetection)

The custom config param in the yaml will have a python script **MMDetection.py**. This script can be configured to use the MMDetection configs

#### Preview of saved MMDetection.py custom config file

In [28]:
file = open('saved_MMDetection.py')
script = file.read()
print(script)

_base_ = '/mmdetection/configs/yolof/yolof_r50_c5_8x8_1x_coco.py'
model=dict(
  bbox_head=dict(num_classes=0))
data=dict(
  train=dict(
    ann_file='',
    img_prefix='',
    classes=''
    ),
  val=dict(
    ann_file='',
    img_prefix='',
    classes=''))
optimizer=dict(
  _delete_=True,
  type='Adam',
  lr=0.0001,
  weight_decay=0.0001)
lr_config = dict(
  _delete_=True,
  policy='CosineAnnealing',
  warmup='linear',
  warmup_iters=1000,
  warmup_ratio=0.1,
  min_lr_ratio=1e-5)
runner = dict(
  _delete_=True,
  type='EpochBasedRunner',
  max_epochs=10)


## Important Note: 
- See [MMDetection](https://mmdetection.readthedocs.io/en/dev/tutorials/config.html) link for general configuration guidelines.
- Here we are starting with the base config for yolof (see [MMDetection code](https://github.com/open-mmlab/mmdetection/blob/main/configs/yolof/yolof_r50-c5_8xb8-1x_coco.py)) 
- Changing the optimizer to Adam, customizing optimizer config. On top of that we are modifying the learning rate config.
- Adopting a cosine annealing policy.
- Setting the number of epochs to 10. 
  - Note the usage of _delete_ above. If not used, the dict that is being defined is merged to the _base_ config; which might define an invalid configuration. 

### Start Model Training

In [ ]:
model_version_id = model.train(yaml_file='saved_mmdetection.yaml')

### Check Model Training Status
Note: If the status code is 'MODEL_TRAINED', then the user can know the Model is Trained and ready to use.

In [ ]:
#To store training logs in a file, fix training_logs param as True
status = model.training_status(version_id=model_version_id,training_logs=False)
print(status)

### Model Prediction
Predicting with the Trained Model.
Note: Refer this [notebook](https://github.com/Clarifai/examples/blob/main/models/model_predict.ipynb) for more info on Model Predict

In [ ]:
IMAGE_PATH = os.path.join(os.getcwd().split('/models')[0],'datasets/upload/image_detection/voc/images/2007_000464.jpg')

prediction_response = model.predict_by_filepath(IMAGE_PATH, input_type="image")

# Get the output
regions = prediction_response.outputs[0].data.regions

In [ ]:
# Display the prediction bboxes
import cv2
from urllib.request import urlopen
import numpy as np

img = plt.imread(IMAGE_PATH)

for region in regions:
    # Accessing and rounding the bounding box values
    top_row = round(region.region_info.bounding_box.top_row, 3) * img.shape[0]
    left_col = round(region.region_info.bounding_box.left_col, 3)* img.shape[1]
    bottom_row = round(region.region_info.bounding_box.bottom_row, 3)* img.shape[0]
    right_col = round(region.region_info.bounding_box.right_col, 3)* img.shape[1]

    cv2.rectangle(img, (int(left_col),int(top_row)), (int(right_col),int(bottom_row)), (36,255,12), 2)

    # Get concept name
    concept_name = region.data.concepts[0].name 

    # Display text 
    cv2.putText(img, concept_name, (int(left_col),int(top_row-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2) 

plt.axis('off')
plt.imshow(img[...,::-1])

## Note

- This Notebook is a demo to get started with Model Training in Clarifai Platform with Python SDK.
- For better accuracy of the Model, Choose your own data and different Templates and Hyperparams.

## Clarifai Resources

**Website**: [https://www.clarifai.com](https://www.clarifai.com/)

**Demo**: [https://clarifai.com/demo](https://clarifai.com/demo)

**Sign up for a free Account**: [https://clarifai.com/signup](https://clarifai.com/signup)

**Developer Guide**: [https://docs.clarifai.com](https://docs.clarifai.com/)

**Clarifai Community**: [https://clarifai.com/explore](https://clarifai.com/explore)

**Python SDK Docs**: [https://docs.clarifai.com/python-sdk/api-reference](https://docs.clarifai.com/python-sdk/api-reference)

---